# UCI MTB DH Data Retrieval

## Setup
#### Import Libraries

If you do not have these libraries available, you should install them using `pip`

```
pip install requests
pip install bs4
pip install pandas
```

In [239]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Widen display area to prevent column wrapping, and always show all columns for debug 

In [240]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

## Config

Which race data are we collecting?

1. Losinj
1. Fort William
1. Leogang
1. Val di Sole
1. Vallnord
1. Mont-Sainte-Anne
1. La Bresse

In [241]:
race = 1
gender = 'f'

#### Data Sources

The UCI Live Timing API contains a lot of data points, but not all the ones we want (speed being the main one missing), and not even all the ones they include on their own PDF which is frustrating.

Similarly, Roots & Rain also has a lot of the data points, but again not all of them; most notably it's missing timing splits 4 and 5.

Therefore we need to pull from both sources and combine the sets.

Here we specify the URLs for both sources from which we will extract our data. The UCI API URL can be found by loading the Live Timing page then using your browser's inspector on the Network tab (in Chrome at least) to see the data feed. As the UCI seems to be using a Single Page Application (SPA) here, it's not straight forward to extract this link automagically.

*Links will be added as data sources become available.*

In [242]:
races = [
    [
        'losinj',
         'http://prod.chronorace.be/api/results/uci/dh/race/20180421_dh/3',
         'https://www.rootsandrain.com/race5897/2018-apr-22-mercedes-benz-uci-world-cup-1-losinj/results/filters/m/',
         'http://prod.chronorace.be/api/results/uci/dh/race/20180421_dh/6',
         'https://www.rootsandrain.com/race5897/2018-apr-22-mercedes-benz-uci-world-cup-1-losinj/results/filters/f/'
    ]
    , [ 'fortbill', '', '' ]
    , [ 'leogang', '', '' ]
    , [ 'valdisole', '', '' ]
    , [ 'vallnord', '', '' ]
    , [ 'msa', '', '' ]
    , [ 'labresse', '', '' ]
]
key = 0 if gender == 'm' else 2
raceName = races[race-1][0]
urlUci = races[race-1][key+1]
urlRoots = races[race-1][key+2]

# UCI API
### Load Data

These two lines make the actual request to the server, and then converts the JSON string response in to a usable list format (deserialization)

In [243]:
r = requests.get( urlUci )
d = r.json()

The API returns with three main sections:

1. `Last Finisher`
 - Racers in order of start time
2. `Results`
 - Racers in finishing rank order
3. `Riders`
 - Personal details on all racers
 
Each contains many data points. To see all the contained data, you can un-comment and execute any of the lines in the next section to explore more.

In [245]:
# display( d )
# display( d['Results'][7] )
# display( d['Riders']['1001'] )
# display( d['Results'][61] )

### Extract Data

Here we iterate over the `Results` sub-set of data to extract the information we care about: basically those that finished the race, some identifying info, and their splits.

If you looked at detail of the returned data set in the last step you might have noticed the rider's name is not stored next to their result, riders are only identified by a reference number. To facilitate our analysis later on it is useful to import each rider's name at this stage by cross-referencing the `Riders` sub-set.

First we find out the last man to drop-in's start number so we can use that to add a reverse order column.

In [ ]:
lastStart = d['Riders'][list(d['Riders'].keys())[-1]]['StartOrder']

We start with an empty list `lst` and in each loop iteration add an entry (actually a dict) to that list for each rider.

In [ ]:
splits = len(d['Results'][0]['Times'] )
lst = []
for idx, row in enumerate( d['Results'] ):
    fin = "Finished" == row['Status']
    res = {
        'rank': row['Position'] if fin else idx+1,
        'name': d['Riders'][str(row['RaceNr'])]['PrintName'],
        'id': row['RaceNr'],
        'uci': d['Riders'][str(row['RaceNr'])]['UciRiderId'],
        'bib': d['Riders'][str(row['RaceNr'])]['RaceNr'],
        'status': row['Status'],
        'speed': np.nan,
        'start': d['Riders'][str(row['RaceNr'])]['StartOrder'],
        'start_rev': lastStart - d['Riders'][str(row['RaceNr'])]['StartOrder'] +1
    }

    # Add all splits to result set
    for split in range( 0, splits ):
        res['split' + str(split+1)] = row['Times'][split]['RaceTime']/1000 if fin else np.nan

    # Append result set to list
    lst.append(res)

This line loads the completed list in to a Pandas dataframe so that we can easily write it out to CSV later on 

In [ ]:
df = pd.DataFrame( lst )

#### Expand Dataset

Calculate and add all the extra columns we need for split and sector differences and their rankings

In [ ]:
for i in range( 1, splits+1 ):
    split = 'split' + str(i)
    sector = split + '_sector'
    df[split + '_rank'] = df[split].rank(method='dense')
    df[split + '_vs_best'] = (df[split] - df[split].min())
    df[split + '_vs_winner'] = (df[split] - df[split][0])
    
    if i > 1:
        df[split + '_sector'] = df[split] - df['split' + str(i-1)]
        df[split + '_sector_rank'] = df[sector].rank(method='dense')
        df[split + '_sector_vs_best'] = (df[sector] - df[sector].min())
        df[split + '_sector_vs_winner'] = (df[sector] - df[sector][0])

We can take a peek at our data at this point to make sure it looks how we expect.

At this point the `speed` column is NaN (Not a Number) for all racers. This will be filled in below.

In [ ]:
display( df.head(10) )

#### Rider Data

Saving the personal information about each racer is much easier as we can just export the entire `Riders` dataset. However, the rows and columns are the wrong way round so the `.T` command *transposes* the information, meaning it basically flips the axes.

In [ ]:
df2 = pd.DataFrame( d['Riders'] )
df2 = df2.T

Here we can glimpse the first few rows of our `DataFrame` and can check the data looks as we expect

In [ ]:
display( df2.head() )

# Roots and Rain
### Load Data

Similar to the UCI api, we make a request to the server with the previously declared `urlRoots` variable. This time however we simply load the content of the response as text which is actually the HTML code of the web page. We do not do have a nice JSON API to read which means we will not deserialize.

Next we invoke a utility called `BeautifulSoup` to help us extract the data from this messy HTML code

In [ ]:
r = requests.post( urlRoots )
c = r.content
soup = BeautifulSoup( c, "html.parser" )

### Extract Data

If you look at the Roots and Rain page you'll see it listed in a tabular format. What we do here is find all the rows of that table so we can extract the information we need.

Specifically we are looking for instances of `tr` (table row), with a class that *begins with* `c-` as this is a common denomenator I discovered when looking through the code with the browser inspector

In [ ]:
rows = soup.find_all( "tr", class_=lambda x: x and 'c-' in x )

Similar to the UCI data set, here we will iterate over each row in our data set--basically each table row from the web page--and extract the bits we need.

Racer speed is the metric we're interested in, but in order to match that to our existing data set we need a corresponding identifier so we also extract the racer licence number as that exists in both sets and we can match them together: it is the *intersect* between both sets of data.

To summarise:
1. Extract licence number and corresponding speed
2. Import speed to existing DataFrame matching racers by licence

The `if` condition in the middle will exit this block of code once we hit the end of the Elite finishers, seeing as that's all we have in our existing data set so can't match anyone else

In [ ]:
for row in rows:
    cells = row.find_all( "td" )

    speed = float(cells[7].text[:5])
    licence = cells[4].text
    bib = int( cells[1].text )
    pos = cells[0].text[8:]
    if "" == pos: break

    # Match rider by UCI licence if present, otherwise fallback to bib
    if len(df2.loc[df2['UciRiderId'] == licence].index.values ):
        rid = int(df2.loc[df2['UciRiderId'] == licence].index.values[0])
    else:
        rid = int( df2.loc[df2['RaceNr'] == bib].index.values[0] )

    # Add speed, and other associated metrics
    df.loc[df['id'] == rid, 'speed'] = speed
    df.loc[df['id'] == rid, 'speed_ms'] = float(speed)*(1000/60/60)
    df.loc[df['id'] == rid, 'speed_ms_vs_best'] = df['speed_ms'].max() - df.speed_ms
    df['speed_rank'] = df.speed.rank(method='dense', ascending=False)

As before, we can take another look at how our data is looking, with the `speed` column now containing data 

In [ ]:
display( df.head() )

# Data Export

All that's left is to save our data to CSV files so we can quickly import it again for analysis and visualization without making constant requests to the online servers. This not only reduces load on the services providing the data, but also allows us to work on our analysis "offline", moreover giving us a local copy in case the results are ever taken down. It's also much quicker to load data this way than constantly hitting online servers.

In [ ]:
event = 'dh'
filePrefix = event + '_' + str(race) + '_' + raceName + '_' + gender
df.to_csv( filePrefix + '.results.csv' )
df2.to_csv( filePrefix + '.racers.csv' )

--- 

## Credits

### Author: Dominic Wrapson


> **@domwrap**
<br>
<img src="https://png.icons8.com/material/24/000000/github-2.png">
<img src="https://png.icons8.com/material/24/000000/stackoverflow.png">
<img src="https://png.icons8.com/material/24/000000/linkedin.png">
<img src="https://png.icons8.com/material/24/000000/windows8.png">
<img src="https://png.icons8.com/ios-glyphs/24/000000/instagram-new.png">
<img src="https://png.icons8.com/material/24/000000/twitter.png">
<a href="https://medium.com/@domwrap"><img src="https://png.icons8.com/material/24/000000/medium-logo.png"></a>
>
> <img src="https://png.icons8.com/material/24/000000/home.png"> http://domwrap.me
>
><img src="https://png.icons8.com/material/24/000000/cycling-mountain-bike.png"> [Hwulex](https://www.pinkbike.com/u/Hwulex/)


---

#### Special Thanks

Mark Shilton for the inspiration
- http://lookatthestats.blogspot.ca
- https://plus.google.com/+MarkShilton
- https://dirtmountainbike.com/author/mrgeekstats


<a href="https://icons8.com">Icon pack by Icons8</a>